### User input

In [ ]:
decimal = '.'                                                                   # decimal of the input file
sep = ';'                                                                       # separator of the input file
input_dir = 'C:/Users/cace0002/AbspectroscoPY/results/df_mediancor_sw_30.0.csv' # input directory
output = 'C:/Users/cace0002/AbspectroscoPY/results/'                            # output directory
dateheadername = 'Timestamp'                                                    # header of the date  
header = 0                                                                      # header row number
dateparsingformat = '%Y-%m-%d %H:%M:%S'                                         # format of the date 
sample_name = 'sw'                                                              # name of the sample

wl0 = 350                                                                       # reference wavelength in the equation 
                                                                                # a0*exp(-S*(x-wl0))+K
startwl = 220                                                                   # starting wavelength
endwl = 737.5                                                                   # ending wavelength
S_f = 1                                                                         # sampling frequency

#To plot a specific exponential fit, the user can modify:
date_interesting = '2018-12-04 08:30:00' 
title1 = 'exponential_fit_'                                                     # title of the exported figure
fig_format = '.tiff'                                                            # format of the exported figure
dpi = 300                                                                       # resolution of the exported figure

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from datetime import datetime
from scipy import interpolate
from sklearn.linear_model import LinearRegression
import scipy as sp                       # To get the exponential fit
from scipy.optimize import curve_fit
from pylab import *

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)      # make sure time column (here index) is using time format
df

### abs_fit_exponential

In [ ]:
def abs_fit_exponential(df_in, 
                        startwl,
                        endwl,
                        wl0,
                        sampling_frequency): 

    '''
    function to get a dataframe of optimised parameters and one standard deviation error when fitting an exponential curve to a time series of absorbance data
    :argument df_in: dataframe in input
    :argument startwl: starting wavelength
    :argument endwl: ending wavelength
    :argument wl0: reference wavelength in the equation a0*exp(-S*(x-wl0))+K
    :argument sampling_frequency: sampling frequency
    :return: dataframe with optimised parameters (df_out1) and with one standard deviation error (df_out2)
    '''
    
    df_exp = df_in.copy()
    df_exp = df_exp.reset_index()
    df_exp = df_exp.T                                                 # transpose the data frame  
    df_exp = df_exp.reset_index()                                     # restore the original index as row
    headers = df_exp.iloc[0]                                          # rename the dates as headers                                     
    df_exp = pd.DataFrame(df_exp.values[1:], columns = headers)       # convert the first row to header
    df_exp = df_exp.rename(columns={"Timestamp": "wl"})               # rename the first column as "wl" = wavelength vector
    df_exp['wl'] = df_exp['wl'].replace({'nm':''}, regex=True)        # remove "nm" from the wavelength vector
    wl = df_exp['wl'].apply(pd.to_numeric)                            # convert the wavelength vector to numeric  

    iteration=len(df_exp.columns)                                     # number of loop iterations
    print('number of iterations',iteration)
    empty_matrix=np.zeros((iteration, 3))                             # create an empty matrix to fill in with the parameters of the exponential fit for each datetime  (a0*np.exp(-S*(x-wl0))+K)
    empty_matrix_2=np.zeros((iteration, 3))                           # create an empty matrix to fill in with the 1 std for the parameters of the exponential fit for each datetime
     
    counter=0
    for i in range(0,iteration, sampling_frequency): 
        counter = i
        print(counter) 
        absorbance = df_exp.iloc[:,i]
        x = wl[(wl >= startwl) & (wl <= endwl)]
        y = absorbance[(wl >= startwl) & (wl <= endwl)]
        sf = interpolate.interp1d(wl, absorbance)
        a0=sf(wl0)
        par_init_vals=[a0,0.02,0.01]                                  # for [a0,S,K]
        par_bounds= ([0, 0, -Inf], [max(y), 1, Inf])                  # for [a0,S,K]

        #1. Find out the best exponential fit to the data 
        popt,pcov=sp.optimize.curve_fit(lambda x,a0,S,K: a0*exp(-S*(x-wl0))+K,  x, y, p0 = par_init_vals, bounds = par_bounds, maxfev = 600) # scipy.optimize.curve_fit uses non-linear least squares to fit a function, f, to data; bounds to constrain the optimization to the region indicated by the lower and upper limit of the a0, S, K parameters; maxfev specifies how many times the parameters for the model that we are trying to fit are allowed to be altered
        #print('popt a0,S,K:',popt)                                   # the best-fit parameters.
        a0=round(popt[0],3)                                           # round the parameter to three digits
        S=round(popt[1],3)
        K=round(popt[2],3)    
        empty_matrix[i, :] = popt
        #print(empty_matrix)
        #print('pcov',pcov)                                           # the estimated covariance of popt. The diagonals provide the variance of the parameter estimate. 
        perr = np.sqrt(np.diag(pcov))                                 # compute one standard deviation errors on the parameters
        #print('perr a0,S,K:',perr)
        a0_std=round(perr[0],3)                                       # round the parameter to three digits
        S_std=round(perr[1],3)
        K_std=round(perr[2],3)
        empty_matrix_2[i, :] = perr
        #print(empty_matrix_2)

    names = [_ for _ in 'aSK']
    exp_par = pd.DataFrame(empty_matrix, index=headers, columns=names) # all the headers, excluding the "wl" header
    #print(exp_par)
    exp_par_no_zero = exp_par[exp_par['S'] != 0]                       # keep only rows in which S is different from zero 
    print('Optimised parameters:', '\n', exp_par_no_zero)
    
    names_std = [_ for _ in 'aSK']
    exp_par_std = pd.DataFrame(empty_matrix_2,index=headers,columns=names_std) 
    #print(exp_par_std)
    exp_par_std_no_zero = exp_par_std[exp_par_std['S'] != 0]           # keep only rows in which S is different from zero 
    print('One standard deviation errors on parameters:', '\n', exp_par_std_no_zero)
    
    df_out1 = exp_par_no_zero
    df_out2 = exp_par_std_no_zero 
    return(df_out1, df_out2) 

In [ ]:
exp_par_no_zero, exp_par_std_no_zero = abs_fit_exponential(df, startwl, endwl, wl0, S_f)
exp_par_no_zero[1:].to_csv(output +'exp_fit_' + str(sample_name) + '_' + str(wl0) + '_' + str(S_f) +'.csv', index = True, sep = ';')
exp_par_std_no_zero[1:].to_csv(output + 'exp_fit_std_' + str(sample_name) + '_' + str(wl0) + '_' + str(S_f) +'.csv', index = True, sep =';')     

In [ ]:
abspy.abs_fit_exponential_plot(df, output, dateparsingformat, date_interesting, startwl, endwl, wl0)